In [ ]:
import umap

import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
rcParams['figure.figsize'] = (12,12)
import seaborn as sns
# 
sc.settings.verbosity =0

import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
from scib_metrics.nearest_neighbors import  pynndescent
import scib_metrics

# import pickle
import muon as mu

import os
compute = True

import numpy as np
import pandas as pd

# Ensure the directory for intermediate and final results exists
if not os.path.exists('CLISI'):
    os.makedirs('CLISI')
if not os.path.exists('Silhouette'):
    os.makedirs('Silhouette')

In [2]:
l1_colors = {
'ATL': '#1f77b4',
  'PT_VCAM1': '#c5b0d5',
 'CNT': '#ff7f0e',
 'DCT': '#279e68',
  'DCT1': '#279e68',
 'DCT2': '#ffb500',
 'DTL': '#d62728',
 'EC': '#aa40fc',
 'ENDO': '#aa40fc',
 'FIB': '#8c564b',
 'IC': '#e377c2',
 'ICA': '#e377c2',
 'ICB': '#7b4f4b',
 'IMM': '#b5bd61',
 'LEUK': '#b5bd61',
 'NEU': '#17becf',
 'PC': '#aec7e8',
 'PEC': '#17becf',
 'POD': '#98df8a',
 'PODO': '#98df8a',
'PT': '#ff9896',
 'PapE': '#c5b0d5',
 'TAL': '#c49c94',
'MES': '#f7b6d2',
 'VSM/P': '#f7b6d2','Unclassified':'#d3d3d3'}

In [3]:
l3_colors = {
'ATL': '#ffff00',
 'B': '#1ce6ff',
 'C-TAL': '#ff34ff',
 'PC': '#aec7e8',
 'CCD-IC-A': '#ff4a46',
 'CCD-PC': '#008941',
 'CNT': '#006fa6',
 'CNT-IC-A': '#a30059',
 'CNT-PC': '#ffdbe5',
 'DCT1': '#7a4900',
 'DCT2': '#0000a6',
 'DTL': '#63ffac',
 'EC-AEA': '#b79762',
 'EC-AVR': '#004d43',
 'EC-DVR': '#8fb0ff',
 'EC-GC': '#997d87',
 'EC-LYM': '#5a0007',
 'EC-PTC': '#809693',
 'FIB': '#6a3a4c',
 'IC-B': '#1b4400',
 'IMCD': '#4fc601',
 'M-FIB': '#3b5dff',
 'M-TAL': '#4a3b53',
 'MAC-M2': '#ff2f80',
 'MAST': '#61615a',
 'MC': '#ba0900',
 'MD': '#6b7900',
 'MDC': '#00c2a0',
 'MYOF': '#ffaa92',
 'N': '#ff90c9',
 'NEU': '#b903aa',
 'NKC/T': '#d16100',
 'OMCD-IC-A': '#ddefff',
 'OMCD-PC': '#000035',
 'PEC': '#7b4f4b',
 'PL': '#a1c299',
 'POD': '#300018',
 'PT-S1/2': '#0aa6d8',
 'PT-S3': '#013349',
 'PapE': '#00846f',
 'REN': '#372101',
 'T': '#ffb500',
 'VSMC': '#c2ffed',
 'VSMC/P': '#a079bf',
 'aFIB': '#cc0744',
 'aPT': '#c0b9b2',
 'aTAL1': '#c2ff99',
 'aTAL2': '#001e09',
 'cDC': '#00489c',
 'cycCNT': '#6f0062',
 'cycDCT': '#0cbd66',
 'cycEC': '#eec3ff',
 'cycMNP': '#456d75',
 'cycMYOF': '#b77b68',
 'cycNKC/T': '#7a87a1',
 'cycPT': '#788d66',
 'dC-IC-A': '#885578',
 'dC-TAL': '#fad09f',
 'dCNT': '#ff8a9a',
 'dDCT': '#d157a0',
 'dEC': '#bec459',
 'dEC-PTC': '#456648',
 'dFIB': '#0086ed',
 'dIMCD': '#886f4c',
 'dM-FIB': '#34362d',
 'dM-TAL': '#b4a8bd',
 'dOMCD-PC': '#00a6aa',
 'dPT': '#452c2c',
 'dVSMC': '#636375',
 'ncMON': '#a3c8c9',
 'pDC': '#ff913f',
 'tPC-IC': '#938a81','Unclassified':'#d3d3d3'}

In [ ]:

def df_aver(adata_obs, ann, ann_path, csv_folder='CLISI', mods=['Seurat_WNN', 'snRNA_Multi'], value_metric='median', integrated=False, threshold=0, min_pop_size=40):
    adata_obs[ann] = adata_obs[ann].tolist()
    adata_obs[ann] = adata_obs[ann].astype('category')
    ct_list = adata_obs[ann].cat.categories
    
    df_averages = pd.DataFrame(index=ct_list, columns=[f'{ann}_{mod}' for mod in mods])
    df_counts = pd.DataFrame(index=ct_list, columns=[f'{ann}_{mod}' for mod in mods])
    
    for mod in mods:
        for ct in ct_list:
            ct_sanitized = ct.replace('/', '_')
            filepath = os.path.join(csv_folder, mod, ann_path)
            temp_df = pd.read_csv(filepath + '.csv', index_col=0)
            
            idxs = adata_obs[ann].isin([ct])
            count = np.sum(idxs)
            df_counts.loc[ct, f'{ann}_{mod}'] = count
            
            if count > min_pop_size:
                if value_metric == 'median':
                    average_score = temp_df.loc[idxs]['binary_LISI'].median()
                elif value_metric == 'mean':
                    average_score = temp_df.loc[idxs]['binary_LISI'].mean()
                else:
                    average_score = np.nan

                if pd.isna(average_score):
                    df_averages.loc[ct, f'{ann}_{mod}'] = 0 
                else:
                    df_averages.loc[ct, f'{ann}_{mod}'] = average_score 
            else:
                print(f'{ct} in {ann}_{mod} is too small')
                df_averages.loc[ct, f'{ann}_{mod}'] = 0
                
    return df_averages, df_counts

    
def binary_LISI(neigh, metadata):
    """
    Compute a binary version of the Local Inverse Simpson Index (LISI).

    :param neigh: A pynndescent object with the 'indices' attribute having shape (n_cells, n_neighbors)
    :param metadata: An array of shape (n_cells,) containing metadata labels for each cell
    :return: A numpy array of shape (n_cells,) with the binary LISI scores
    """
    n_cells = neigh.indices.shape[0]
    binary_lisi_scores = np.zeros(n_cells)

    for i in range(n_cells):
        center_label = metadata[i]
        neighbor_indices = neigh.indices[i]
        neighbor_labels = metadata[neighbor_indices]

        # Count how many neighbors have the same label as the center cell
        same_label_count = np.sum(neighbor_labels == center_label)
        
        # Compute the proportion of same label in the neighborhood
        binary_lisi_scores[i] = same_label_count / len(neighbor_indices)

    return binary_lisi_scores

def result_exists(mod, i, ann, n_size, ct):
    ct_sanitized = ct.replace('/', '_')  # Sanitize cell type name for use in filename
    filename = f'CLISI_csv_PARTIAL/scIB_{mod}_{i}_{ann}_BINARY_{n_size}_{ct_sanitized}.csv'
    return os.path.exists(filename), filename

def check_dir(filename):
    directory = os.path.dirname(filename)
    # Create the directory and any intermediate directories, if they don't exist
    os.makedirs(directory, exist_ok=True)

    
def save_binary_lisi_scores(binary_lisi_scores, filename, valid_indices):
    temp_df = pd.DataFrame(binary_lisi_scores, index=valid_indices, columns=['binary_LISI'])
    print(temp_df.shape)

    temp_df.to_csv(filename)

def save_scores(scores, filename, valid_indices, col_name = 'score'):
    temp_df = pd.DataFrame(scores, index=valid_indices, columns=[col_name])
    print(temp_df.shape)
    check_dir(filename)
    temp_df.to_csv(filename)


# COMPUTING CLISI - RNA


In [6]:
folder = '../HORIZONTAL_RNA/objects/byTech/'
save_key = i = 'scVI'
ann_l = ['Deepscore_HCA_l1_Clean', 'Deepscore_HCA_l3_Clean']
n_size = 50
for object in ['scRNA.h5ad', 'scRNA5p.h5ad','snRNA.h5ad']:
    if object.startswith('.') == False:
        object_san = object.replace('.h5ad','')
        try:
            os.mkdir(f'CLISI/{object_san}')
        except:
            print('CLISI Dir already existis.')
        adata = sc.read(f'../HORIZONTAL_RNA/objects/byTech/{object}')
        adata.obs.to_csv(f'obs/{object_san}.csv')
        neigh = pynndescent(adata.obsm[i], n_neighbors=n_size)

        for ann in ann_l:
            # Perform pynndescent computation
            binary_lisi_scores = binary_LISI(neigh, adata.obs[ann])
            # Save the computed binary LISI scores to a CSV file
            filename = f'CLISI/{object_san}/{ann}.csv'
            save_binary_lisi_scores(binary_lisi_scores, filename,adata.obs.index)


            silhouette_scores = scib_metrics.utils.silhouette_samples(adata.obsm[i], adata.obs[ann])
            # Save the computed binary LISI scores to a CSV file
            filename = f'Silhouette/{object_san}/{ann}.csv'
            save_scores(silhouette_scores, filename,adata.obs.index, col_name = 'Silhouette')

            silhouette_scores = scib_metrics.utils.silhouette_samples(adata.obsm[i], adata.obs[ann])
            # Save the computed binary LISI scores to a CSV file
            filename = f'Silhouette/{object_san}/{ann}.csv'
            save_scores(silhouette_scores, filename,adata.obs.index, col_name = 'Silhouette')

CLISI Dir already existis.


/tmp/ipykernel_11475/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_11475/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(35513, 1)
(35513, 1)
(35513, 1)


/tmp/ipykernel_11475/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_11475/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(35513, 1)
(35513, 1)
(35513, 1)
CLISI Dir already existis.


...

(7813, 1)
(7813, 1)
(7813, 1)
CLISI Dir already existis.


/tmp/ipykernel_11475/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_11475/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(53799, 1)
(53799, 1)
(53799, 1)


/tmp/ipykernel_11475/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_11475/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(53799, 1)
(53799, 1)
(53799, 1)


In [7]:
folder = '../HORIZONTAL_RNA/objects/'
save_key = i = 'scVI'
ann_l = ['Deepscore_HCA_l1_Clean', 'Deepscore_HCA_l3_Clean']
n_size = 50
object = 'RNA_STWG_final.h5ad'
try:
    os.mkdir(f'CLISI/{object_san}')
except:
    print('CLISI Dir already existis.')
adata = sc.read(f'../HORIZONTAL_RNA/objects/{object}')
adata.obs.to_csv('obs/RNA_Integration.csv')
print(adata.obs)
neigh = pynndescent(adata.obsm[i], n_neighbors=n_size)

for ann in ann_l:
    # Perform pynndescent computation
    binary_lisi_scores = binary_LISI(neigh, adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'CLISI/RNA_STWG_final/{ann}.csv'
    save_binary_lisi_scores(binary_lisi_scores, filename,adata.obs.index)

    silhouette_scores = scib_metrics.utils.silhouette_samples(adata.obsm[i], adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'Silhouette/RNA_STWG_final/{ann}.csv'
    save_scores(silhouette_scores, filename,adata.obs.index, col_name = 'Silhouette')

CLISI Dir already existis.
                              sample    batch  n_counts  n_genes_by_counts  \
ACTCACCTCGCTCCAT-11-snRNA     lib_55    snRNA     485.0                370   
ACTTACAAGGTCAAAG-6-snRNA      lib_34    snRNA    1904.0               1281   
AGCACTTAGTTCCCGT-1-snRNA      lib_09    snRNA     686.0                382   
AGCCTAAGTCAAGTGC-6-snRNA      lib_34    snRNA     232.0                205   
ATGAAGCCACCTCGCT-12-snRNA     lib_56    snRNA     304.0                255   
...                              ...      ...       ...                ...   
TTTCCTCTCGGAAACG-1-3-scRNA5p  lib_09  scRNA5p   14882.0               3069   
TTTGCGCAGTGGCACA-1-3-scRNA5p  lib_09  scRNA5p    1799.0                599   
TTTGGTTCAATCCGAT-1-3-scRNA5p  lib_09  scRNA5p    1563.0                773   
TTTGGTTTCAAAGTAG-1-3-scRNA5p  lib_09  scRNA5p    1460.0                698   
TTTGTCAGTCACAAGG-1-3-scRNA5p  lib_09  scRNA5p    1628.0                759   

                              total_

/tmp/ipykernel_17646/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_17646/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(97125, 1)
(97125, 1)


/tmp/ipykernel_17646/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_17646/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(97125, 1)
(97125, 1)


# ATAC

In [8]:
folder = '../HORIZONTAL_ATAC/objects/'
save_key = i = 'X_spectral_mnn'
ann_l = ['Deepscore_HCA_l1_Clean', 'Deepscore_HCA_l3_Clean']
n_size = 50

adata = sc.read(f'../HORIZONTAL_ATAC/objects/atac_full.h5ad')
adata.obs.to_csv('obs/atac.csv')
adata = adata[~adata.obs['Deepscore_HCA_l1_Clean'].isin([pd.NA])]

neigh = pynndescent(adata.obsm[i], n_neighbors=n_size)

for ann in ann_l:
    # Perform pynndescent computation
    binary_lisi_scores = binary_LISI(neigh, adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'CLISI/Spectral_Atac_mnn/{ann}.csv'
    save_binary_lisi_scores(binary_lisi_scores, filename,adata.obs.index)

    silhouette_scores = scib_metrics.utils.silhouette_samples(adata.obsm[i], adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'Silhouette/Spectral_Atac_mnn/{ann}.csv'
    save_scores(silhouette_scores, filename,adata.obs.index, col_name = 'Silhouette')

/tmp/ipykernel_14995/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_14995/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(37717, 1)
(37717, 1)


/tmp/ipykernel_14995/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_14995/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(37717, 1)
(37717, 1)


# VERTICAL

In [10]:
import muon as mu
folder = '../VERTICAL_INTEGRATION/objects/'
save_key = i = 'Spectral_Multi_mnn'
ann_l = ['Deepscore_HCA_l1_Clean', 'Deepscore_HCA_l3_Clean']
n_size = 50

adata = mu.read(f'../VERTICAL_INTEGRATION/objects/mdata_final_comparison.h5mu')
adata.obs.to_csv('obs/Vertical.csv')
adata = adata[~adata.obs['Deepscore_HCA_l1_Clean'].isin([pd.NA])]

neigh = pynndescent(adata.obsm[i], n_neighbors=n_size)

for ann in ann_l:
    # Perform pynndescent computation
    binary_lisi_scores = binary_LISI(neigh, adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'CLISI/Spectral_Multi_mnn/{ann}.csv'
    save_binary_lisi_scores(binary_lisi_scores, filename,adata.obs.index)

    silhouette_scores = scib_metrics.utils.silhouette_samples(adata.obsm[i], adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'Silhouette/Spectral_Multi_mnn/{ann}.csv'
    save_scores(silhouette_scores, filename,adata.obs.index, col_name = 'Silhouette')

/home/macera/.conda/envs/scanpy_basic/lib/python3.9/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/macera/.conda/envs/scanpy_basic/lib/python3.9/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/tmp/ipykernel_17646/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_17646/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_i

(37717, 1)
(37717, 1)


/tmp/ipykernel_17646/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_17646/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(37717, 1)
(37717, 1)


# MOSAIC

In [5]:
adata = mu.read('../MOSAIC_INTEGRATION/objects/multiVI.h5ad')
pd.DataFrame(adata.obsm['MultiVI'], index = adata.obs.index).to_csv('../MOSAIC_INTEGRATION/csv/multiVI_save.csv')

atac_ann = pd.read_csv('../DEEPSCORE/csv/atac_unpaired_annotations.csv', index_col=0)
atac_ann.index = [i+'-snMulti_accessibility' for i in atac_ann.index]
atac_ann = atac_ann[atac_ann.index.isin(adata.obs.index)]

adata.obs['Deepscore_HCA_l1_Clean'] = adata.obs['Deepscore_HCA_l1_Clean'].tolist()
adata.obs['Deepscore_HCA_l3_Clean'] = adata.obs['Deepscore_HCA_l3_Clean'].tolist()
adata.obs.loc[atac_ann.index,'Deepscore_HCA_l1_Clean'] = atac_ann['deepscore_L1_full_annot']
adata.obs.loc[atac_ann.index,'Deepscore_HCA_l3_Clean'] = atac_ann['deepscore_L2_ver3_full_annot']

adata.obs.to_csv('obs/MultiVI.csv')
folder = '../MOSAIC_INTEGRATION/csv/MultiVI'
save_key = i = 'MultiVI'
ann_l = ['Deepscore_HCA_l1_Clean', 'Deepscore_HCA_l3_Clean']
n_size = 50
adata = adata[~adata.obs['Deepscore_HCA_l1_Clean'].isin([pd.NA])]
neigh = pynndescent(adata.obsm[i], n_neighbors=n_size)

for ann in ann_l:
    # Perform pynndescent computation
    binary_lisi_scores = binary_LISI(neigh, adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'CLISI/{i}/{ann}.csv'
    save_binary_lisi_scores(binary_lisi_scores, filename,adata.obs.index)

    silhouette_scores = scib_metrics.utils.silhouette_samples(adata.obsm[i], adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'Silhouette/MultiVI/{ann}.csv'
    save_scores(silhouette_scores, filename,adata.obs.index, col_name = 'Silhouette')

/tmp/ipykernel_17739/1660066902.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_17739/1660066902.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(119744, 1)


# DIAGONAL

In [7]:
adata = mu.read('../../../../beegfs/CZI/Downstream/DIAGONAL_Integration/objects/GLUE_comb_emb_FINAL.h5ad')
adata.obs.to_csv('obs/GLUE_comb.csv')
pd.DataFrame(adata.obsm['X_glue'], index = adata.obs.index).to_csv('../DIAGONAL_INTEGRATION/csv/GLUE.csv')
# folder = '../DIAGONAL_INTEGRATION/csv/GLUE'
save_key = i = 'X_glue'
ann_l = ['Deepscore_HCA_l1_Clean', 'Deepscore_HCA_l3_Clean']
n_size = 50
adata = adata[~adata.obs['Deepscore_HCA_l1_Clean'].isin([pd.NA])]

neigh = pynndescent(adata.obsm[i], n_neighbors=n_size)

for ann in ann_l:

    # Perform pynndescent computation
    binary_lisi_scores = binary_LISI(neigh, adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'CLISI/GLUE/{ann}.csv'
    save_binary_lisi_scores(binary_lisi_scores, filename,adata.obs.index)

    silhouette_scores = scib_metrics.utils.silhouette_samples(adata.obsm[i], adata.obs[ann])
    # Save the computed binary LISI scores to a CSV file
    filename = f'Silhouette/GLUE/{ann}.csv'
    save_scores(silhouette_scores, filename,adata.obs.index, col_name = 'Silhouette')

/home/macera/.conda/envs/scanpy_basic/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/tmp/ipykernel_14995/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_14995/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(134842, 1)
(134842, 1)


/tmp/ipykernel_14995/143264723.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_label = metadata[i]
/tmp/ipykernel_14995/143264723.py:142: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = metadata[neighbor_indices]


(134842, 1)
(134842, 1)
